# 目录
- [关于 Guidance](../../../../code/01.Introduce)
- [设置](../../../../code/01.Introduce)
- [无约束生成](../../../../code/01.Introduce)
- [Phi 3 的发言](../../../../code/01.Introduce)
- [正则表达式](../../../../code/01.Introduce)
- [选择](../../../../code/01.Introduce)
- [思维链](../../../../code/01.Introduce)
- [JSON 生成](../../../../code/01.Introduce)
- [HTML 生成](../../../../code/01.Introduce)


# 关于 Guidance
Guidance 是一个经过验证的开源 Python 库，用于控制任何语言模型（LM）的输出。通过一次 API 调用，您可以在 Python 中表达模型必须遵循的精确程序约束，并生成 JSON、Python、HTML、SQL 或任何用例所需的结构化输出。

Guidance 不同于传统的提示技术。它通过在推理层逐个引导模型的 token 来强制执行约束，从而生成更高质量的输出，并在处理高度结构化场景时将成本和延迟减少多达 30–50%。

要了解有关 Guidance 的更多信息，请访问 [GitHub 上的公共仓库](https://github.com/guidance-ai/guidance) 或观看 Microsoft Build 的 [Guidance 分组会议](https://www.youtube.com/watch?v=qXMNPVVlCMs)。


# 设置
1. 使用 `pip install guidance --pre` 安装 Guidance
2. 在 Azure 部署一个 Phi 3.5 mini 端点，访问 https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml 并点击“部署”按钮
3. 将端点的 API 密钥存储在名为 `AZURE_PHI3_KEY` 的环境变量中，并将 URL 存储在名为 `AZURE_PHI3_URL` 的环境变量中


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# 无约束生成
可以使用 `gen()` 函数生成没有任何约束的文本。这与使用模型而不使用 Guidance 是相同的。

## 聊天格式
与许多聊天模型类似，Phi-3 期望用户和助手之间的消息以特定格式呈现。Guidance 支持 Phi-3 的聊天模板，并将为您管理聊天格式。要创建聊天轮次，请将对话的每一部分放在 `with user()` 或 `with assistant()` 块中。可以使用 `with system()` 块来设置系统消息。


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## 节省令牌
在高度结构化的场景中，Guidance可以跳过不必要的令牌，仅生成所需的令牌，从而提升性能、提高效率并节省API成本。生成的令牌在此笔记本中以高亮背景显示。强制生成的令牌则不带高亮显示，其成本与输入令牌相同，估算为输出令牌成本的三分之一。

*注意：* 第一个没有约束的生成示例无法强制任何令牌，因为我们没有提供任何约束条件。


# 为 Phi 3 发声  
通过 Guidance，你可以轻松地将文本注入模型的响应中。如果你希望引导模型的输出朝某个特定方向发展，这将非常有用。


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# 使用正则表达式进行约束
在前面的例子中，Phi 3在回答问题并给出`Canberra`后，提供了后续的解释。为了将模型的输出限制为仅一个单词，可以使用正则表达式。


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

使用正则表达式，仅生成单词`Canberra`。


# 从多个选项中进行选择
当已知一些可能的选项时，可以使用 `select()` 函数让模型从选项列表中进行选择。


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

使用 `select()` 时，仅生成了标记 `Can`。由于 `Canberra` 是唯一可能完成响应的选项，其余标记被强制生成。


# 思维链
思维链是一种技术，可以通过鼓励模型逐步处理问题来提高输出质量。通常，为了得到最终答案，需要多次提示。首先，指示模型逐步思考。然后，再次提示模型提供最终答案。使用标准聊天推理API时，这需要进行两次API调用，模型生成的“思维链”会被收费两次——一次是模型生成时作为输出令牌收费，另一次是第二次调用时作为输入令牌收费。而使用Guidance，整个多步骤过程会作为单次API调用的一部分进行处理和收费，从而降低成本和延迟。


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON 生成
Guidance 可用于确保生成符合 JSON schema 或 pydantic 模型的 JSON，例如这里展示的用户配置文件 schema。


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML生成

Guidance也可以用于生成代码，并遵循编程语言的语法要求。在本节中，我们将创建一个简单的Guidance程序，用于编写非常基础的HTML网页。

我们会将网页分解成较小的部分，每个部分都有其独立的Guidance函数。然后在最终的函数中将这些部分组合起来，生成一个HTML网页。
接着，我们会在Azure AI中运行这个函数，使用支持Guidance的模型。

*注意:* 这不会是一个功能齐全的HTML生成器；目标是展示如何根据个人需求创建结构化输出。

首先，我们从Guidance中导入所需内容：


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML网页具有高度结构化，我们将使用Guidance来“强制”页面的这些部分。  
当我们明确要求模型提供文本时，我们需要确保它不包含任何可能是标签的内容——也就是说，我们必须排除“<”和“>”字符。


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

然后我们可以使用此函数在任意HTML标签内生成文本：


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

现在，让我们创建页面标题。  
作为其中的一部分，我们需要生成一个页面标题：


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML页面的主体将会包含标题和段落。  
我们可以定义一个函数来完成每项任务：


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

现在定义 HTML 主体的函数。  
这里使用 `select()` 并设置 `recurse=True` 来生成多个标题和段落：


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

接下来，我们来看生成完整HTML页面的函数。  
我们先添加HTML起始标签，然后生成头部，再生成主体，最后添加结束HTML标签：


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

我们提供了一个用户友好的封装器，它将允许我们：
- 设置生成的温度
- 从 Model 对象中捕获生成的页面


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

然后我们可以将输出写入文件：


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

并[查看结果](../../../../code/01.Introduce/sample_page.html)。



---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
